In [1]:
import pandas as pd
from crewai import Agent, Task
from transformers import pipeline



In [2]:
# Load a Hugging Face model (generic Ollama-style model; replace with the actual model you're using)
llm_model = pipeline("text-generation", model="gpt2")  # Replace with the correct model name

# Define the Banking Data Analyst agent
banking_analyst = Agent(
    role='Banking Data Analyst',
    goal='Analyze customer transactions and identify key investment patterns',
    verbose=True,
    memory=True,
    backstory="An expert in banking and customer data, focused on extracting valuable insights from transactions.",
    tools=None,  # No tools, logic defined within the agent's LLM
    allow_delegation=False,
    llm=llm_model
)



C:\Users\sudes\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transp

In [3]:
# Define the Reporting Agent
report_writer = Agent(
    role='Report Writer',
    goal='Summarize banking data analysis into a report for presentation',
    verbose=True,
    memory=True,
    backstory="A data-driven writer skilled in transforming banking data into comprehensive reports.",
    tools=None,  # No tools, logic defined within the agent's LLM
    allow_delegation=False,
    llm=llm_model
)



In [4]:
# Sample banking data generation function (same as previous)
def generate_banking_data():
    # Sample account-level data
    account_data = {
        'customer_name': ['Alice', 'Bob', 'Charlie', 'David'],
        'customer_id': [1, 2, 3, 4],
        'pr_number': ['PR001', 'PR002', 'PR003', 'PR004'],
        'income': [50000, 60000, 70000, 80000],
        'age': [30, 40, 35, 50]
    }

    # Sample transaction-level data
    transaction_data = {
        'ucic_id': [101, 102, 103, 104, 105, 106],
        'customer_id': [1, 1, 2, 3, 4, 4],
        'category': ['electronics', 'investments', 'investments', 'groceries', 'investments', 'electronics'],
        'transaction_amount': [200, 1500, 2000, 300, 2500, 400],
        'credit_debit': ['debit', 'credit', 'credit', 'debit', 'credit', 'debit']
    }

    # Create DataFrames
    account_df = pd.DataFrame(account_data)
    transaction_df = pd.DataFrame(transaction_data)

    return account_df, transaction_df





In [5]:
# Task for banking data analysis
def perform_banking_analysis(account_df, transaction_df):
    analysis_task = Task(
        description="Analyze customer transactions and identify key investment patterns.",
        expected_output="A detailed report summarizing key insights from banking data, focusing on investment transactions.",
        tools=None,  # No tools required
        agent=banking_analyst
    )

    # Sample analysis logic to filter and summarize investment transactions
    investment_transactions = transaction_df[transaction_df['category'] == 'investments']
    investment_summary = investment_transactions.groupby('customer_id').sum(numeric_only=True)

    # Generate a summary using the LLM pipeline
    analysis_report = analysis_task.agent.llm(f"Analyze the following investment transactions and provide insights:\n{investment_summary.to_string()}")[0]['generated_text']
    
    return analysis_report




In [6]:
# Task for writing a report based on analysis
def perform_report_writing(analysis_report):
    write_task = Task(
        description="Summarize the analysis into a comprehensive banking data report.",
        expected_output="A detailed report summarizing the key insights from the analysis of customer investment data.",
        tools=None,  # No tools required
        agent=report_writer,
        async_execution=False,
        output_file='banking_data_report.md'  # Save the output to a markdown file
    )

    # Generate the report using the LLM pipeline
    report_content = write_task.agent.llm(f"Write a detailed report based on the following analysis:\n{analysis_report}")[0]['generated_text']

    # Save the report to a file
    with open(write_task.output_file, 'w') as file:
        file.write(report_content)
    
    return report_content




In [7]:
# Example usage
account_df, transaction_df = generate_banking_data()
analysis_report = perform_banking_analysis(account_df, transaction_df)
final_report = perform_report_writing(analysis_report)

print("\nBanking Data Analysis Report:")
print(final_report)


TypeError: 'LLM' object is not callable